In [1]:
from datasets import load_dataset
from transformers import DistilBertTokenizer
dataset = load_dataset('tweet_eval','irony')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [2]:
dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2862
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 784
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 955
    })
})

In [3]:
tokenizer

DistilBertTokenizer(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [4]:
import re
import unidecode
def clean_keywords(keyword):
    cleaned = re.sub(r'%20', ' ', keyword)
    cleaned = unidecode.unidecode(keyword)
    cleaned = re.sub(r"[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n -' ]"," ",keyword)
    return cleaned

In [5]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [6]:
tokenized_datasets["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 2862
})

In [7]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2862
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 784
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 955
    })
})

In [8]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
num_labels=2)
training_args = TrainingArguments(
output_dir="./results",
# Directory for saving outputs
learning_rate=5e-5,
# Learning rate for optimization
per_device_train_batch_size=16,
# Batch size for training
per_device_eval_batch_size=16,
# Batch size for evaluation
num_train_epochs=3,
# Number of training epochs
weight_decay=0.01,
# Weight decay for regularization
evaluation_strategy="epoch",
save_strategy = "no"
# Evaluation is done at the end of each epoch
)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]
val_dataset = tokenized_datasets["validation"]
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=eval_dataset,
compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids,
np.argmax(p.predictions,
axis=1))}
)

2024-03-08 22:40:33.277237: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 22:40:33.277280: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 22:40:33.278907: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 22:40:34.324905: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']

In [9]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [9]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [12]:
eval_results = trainer.evaluate(val_dataset)
print(eval_results)

{'eval_loss': 0.6933612823486328, 'eval_accuracy': 0.5225130890052356, 'eval_runtime': 16.3283, 'eval_samples_per_second': 58.488, 'eval_steps_per_second': 3.675}


In [10]:
from ray import tune
def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "per_device_train_batch_size": tune.choice([1,4 ,8 ,16 ]),
        "num_train_epochs": tune.choice([1,3,5]),

        "weight_decay": tune.loguniform(1e-4, 0.1)
    }

In [11]:
small_train_dataset = tokenized_datasets["train"].shard(index=1, num_shards=10) 
small_eval_dataset = tokenized_datasets["test"].shard(index=1, num_shards=10) 


In [12]:
num_labels = 2
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score
def model_init():
    return  DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=num_labels)

In [13]:




num_labels = num_labels
trainer = Trainer(

    model_init= model_init,

    args=training_args,

    train_dataset=small_train_dataset,

    eval_dataset=small_eval_dataset,

    compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids,
np.argmax(p.predictions,
axis=1))}

    

  
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
best_trial = trainer.hyperparameter_search(

    direction="maximize",

    backend="ray",

    hp_space=ray_hp_space,

    n_trials=30

)

/home/ai/anaconda3/lib/python3.11/subprocess.py:1883: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
2024-03-07 01:06:38,165	INFO worker.py:1724 -- Started a local Ray instance.
2024-03-07 01:06:39,741	INFO tune.py:220 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2024-03-07 01:06:39,745	INFO tune.py:583 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-03-07 01:06:39 (running for 00:00:00.20)
Using FIFO scheduling algorithm.
Logical resource usage: 0/12 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_01-06-39
Number of trials: 16/30 (16 PENDING)
+------------------------+----------+-------+-----------------+--------------------+------------------------+----------------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |
|                        |          |       |                 |                    |                ch_size |                |
|------------------------+----------+-------+-----------------+--------------------+------------------------+----------------|
| _objective_9deaf_00000 | PENDING  |       |     5.61152e-06 |                  5 |                      1 |    0.015703    |
| _objective_9deaf_00001 | PENDING  |       |     1.57513e-05 |                  3

(_objective pid=13637) 2024-03-07 01:06:45.617575: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(_objective pid=13637) 2024-03-07 01:06:45.617628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(_objective pid=13637) 2024-03-07 01:06:45.618920: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(_objective pid=13637) 2024-03-07 01:06:46.723805: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(_objective pid=13637) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased 

== Status ==
Current time: 2024-03-07 01:06:50 (running for 00:00:10.34)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_01-06-39
Number of trials: 17/30 (16 PENDING, 1 RUNNING)
+------------------------+----------+------------------+-----------------+--------------------+------------------------+----------------+
| Trial name             | status   | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |
|                        |          |                  |                 |                    |                ch_size |                |
|------------------------+----------+------------------+-----------------+--------------------+------------------------+----------------|
| _objective_9deaf_00000 | RUNNING  | 172.19.0.1:13637 |     5.61152e-06 |                  5 |                      1 |    0.015703    |
| _objective

  5%|▌         | 75/1435 [00:05<01:44, 13.05it/s]


...

== Status ==
Current time: 2024-03-07 01:39:58 (running for 00:33:19.06)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_01-06-39
Number of trials: 30/30 (1 RUNNING, 29 TERMINATED)
+------------------------+------------+------------------+-----------------+--------------------+------------------------+----------------+-------------+
| Trial name             | status     | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |   objective |
|                        |            |                  |                 |                    |                ch_size |                |             |
|------------------------+------------+------------------+-----------------+--------------------+------------------------+----------------+-------------|
| _objective_9deaf_00029 | RUNNING    | 172.19.0.1:20104 |     1.32603e-05 |       

100%|██████████| 108/108 [00:43<00:00,  2.65it/s]
(_objective pid=20104) 
  0%|          | 0/5 [00:00<?, ?it/s]
(_objective pid=20104) 
 40%|████      | 2/5 [00:00<00:00,  7.27it/s]
(_objective pid=20104) 
 60%|██████    | 3/5 [00:00<00:00,  5.14it/s]
(_objective pid=20104) 
 80%|████████  | 4/5 [00:00<00:00,  4.46it/s]


(_objective pid=20104) {'eval_loss': 0.6653863787651062, 'eval_accuracy': 0.6075949367088608, 'eval_runtime': 1.366, 'eval_samples_per_second': 57.833, 'eval_steps_per_second': 3.66, 'epoch': 3.0}
(_objective pid=20104) {'train_runtime': 44.6254, 'train_samples_per_second': 19.294, 'train_steps_per_second': 2.42, 'train_loss': 0.6484169430202908, 'epoch': 3.0}


(_objective pid=20104) 
                                                 
100%|██████████| 108/108 [00:44<00:00,  2.42it/s]
2024-03-07 01:40:02,665	INFO tune.py:1042 -- Total run time: 2002.92 seconds (2002.86 seconds for the tuning loop).


== Status ==
Current time: 2024-03-07 01:40:02 (running for 00:33:22.88)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_01-06-39
Number of trials: 30/30 (30 TERMINATED)
+------------------------+------------+------------------+-----------------+--------------------+------------------------+----------------+-------------+
| Trial name             | status     | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |   objective |
|                        |            |                  |                 |                    |                ch_size |                |             |
|------------------------+------------+------------------+-----------------+--------------------+------------------------+----------------+-------------|
| _objective_9deaf_00000 | TERMINATED | 172.19.0.1:13637 |     5.61152e-06 |                  

In [16]:
best_trial

BestRun(run_id='9deaf_00011', objective=0.6708860759493671, hyperparameters={'learning_rate': 4.1380401125610247e-05, 'per_device_train_batch_size': 1, 'num_train_epochs': 3, 'weight_decay': 0.00019634341572933355}, run_summary=<ray.tune.analysis.experiment_analysis.ExperimentAnalysis object at 0x7fcf20924c50>)

In [11]:

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]
val_dataset = tokenized_datasets["validation"]
trainer = Trainer(

model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=eval_dataset,
compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids,
np.argmax(p.predictions,
axis=1))}
)

In [18]:
for n, v in best_trial.hyperparameters.items():
    setattr(trainer.args, n, v)



In [13]:
trainer.args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,


In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.379700,1.269484,0.557398
2,1.368200,1.362679,0.626276
3,1.237300,1.429887,0.636480


TrainOutput(global_step=8586, training_loss=1.2745887468445976, metrics={'train_runtime': 678.0459, 'train_samples_per_second': 12.663, 'train_steps_per_second': 12.663, 'total_flos': 1137365084860416.0, 'train_loss': 1.2745887468445976, 'epoch': 3.0})

In [15]:
trainer.save_model('/home/ai/Downloads/results/last-distilbert-tweet')

In [16]:
predictions = trainer.evaluate(train_dataset)
print(predictions)

{'eval_loss': 1.2595258951187134, 'eval_accuracy': 0.6767994409503844, 'eval_runtime': 48.9226, 'eval_samples_per_second': 58.501, 'eval_steps_per_second': 3.659, 'epoch': 3.0}


In [17]:
predictions = trainer.evaluate(eval_dataset)
print(predictions)

{'eval_loss': 1.4298874139785767, 'eval_accuracy': 0.6364795918367347, 'eval_runtime': 13.4079, 'eval_samples_per_second': 58.473, 'eval_steps_per_second': 3.655, 'epoch': 3.0}


In [18]:
predictions = trainer.evaluate(val_dataset)
print(predictions)

{'eval_loss': 1.5867059230804443, 'eval_accuracy': 0.5937172774869109, 'eval_runtime': 16.3848, 'eval_samples_per_second': 58.286, 'eval_steps_per_second': 3.662, 'epoch': 3.0}


In [21]:
predictions = trainer.predict(val_dataset)

In [33]:
predictions.predictions

array([[ 2.3662627, -1.8834808],
       [-1.7673728,  2.0297384],
       [-1.7528889,  2.0183425],
       ...,
       [ 2.0480433, -1.6634706],
       [ 2.2471013, -1.8201613],
       [ 2.302975 , -1.8552132]], dtype=float32)

In [38]:
import torch 
predictions.predictions
pre = torch.argmax(torch.tensor(predictions.predictions), dim=1)

In [26]:
true_labels = val_dataset['label']
true_labels

[1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,


In [39]:

from sklearn.metrics import confusion_matrix
# Generate predictions, then create and print the confusion matrix
confusion_mat = confusion_matrix(true_labels, pre)
print(confusion_mat)

[[320 179]
 [209 247]]


In [40]:
predictions = trainer.predict(train_dataset)

In [41]:
predictions.predictions

array([[-1.8230109,  2.0972376],
       [-1.749383 ,  2.0101566],
       [ 2.2912974, -1.8489134],
       ...,
       [ 2.113403 , -1.7203505],
       [ 2.2098367, -1.7940092],
       [ 2.242878 , -1.8170371]], dtype=float32)

In [46]:
import torch 
predictions.predictions
pre = torch.argmax(torch.tensor(predictions.predictions), dim=1)
pre

tensor([1, 1, 0,  ..., 0, 0, 0])

In [48]:
true_labels = train_dataset['label']
true_labels

[1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,


In [49]:

from sklearn.metrics import confusion_matrix
# Generate predictions, then create and print the confusion matrix
confusion_mat = confusion_matrix(true_labels, pre)
print(confusion_mat)

[[1030  387]
 [ 538  907]]


In [88]:
import random


In [78]:
import random

num_examples = 10
picks = []
lable = []
input = []
prediction = [] 
for i in range(num_examples):
        pick = random.randint(0, len(dataset['validation'])-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
        input.append(dataset['validation'][picks[i]]['text'])
        #input_token = tokenizer(dataset['validation'][picks[i]]['text'])
        lable.append(dataset['validation'][picks[i]]['label'])
        encoded_input = tokenizer(dataset['validation'][pick]['text'],return_tensors='pt').to(model.device)
        prediction.append(torch.argmax(trainer.model.forward(**encoded_input).logits, dim=1)[0].cpu().item())
    
    
    


In [84]:
print(input)
print(lable,"lable")
print(prediction,"prediction")



['Women do some crazy ass shit just to be with a guy and the least they can do for her is break her heart!| all but some.', "@user @user @user @user @user Probably folks like #FullMcIntosh who don't buy games.  #gamergate", 'MORON, no one has ever said a ball is square #STRAWMAN @user @user', "girls put so much effort in dressing nice for school..I'm over here in my oversized sweats and vans #maybetomorrow", "Bruh. I just realized I'm wearing her shirt. The chick who dumped me. Lmfao", 'whoever runs Yeovil Town FC twitter account should be fired. their starting line up looks like a mxit fantasy league team', "It's funny how you can sit for 15minutes And not be noticed.  #MTVStars Taylor Swift", 'you know what would make break 1000000x times better?||not having practice.', "They decided 2 change King's trunks 2 gold, his logos 2 green & his tights to blue. How creative. .", '@user Hey, now. Her profile says "I Heart Jesus" so she can\'t be all bad.  @user @user']
[0, 1, 0, 1, 1, 0, 1, 0